In [ ]:
import json
import os

import requests

from oddstracker.config import DATA_DIR
from oddstracker.domain.providers import PROVIDER


In [ ]:
resp = requests.get(PROVIDER.nfl_url, params=PROVIDER.qparams())


In [ ]:
import json

from oddstracker.domain.model.sportsbetting2 import SportsEventInfo

resp = None
#
path = "/home/maxo/code/oddstracker/data/kambi-single.json"
with open(path, encoding="utf-8") as f:
    resp = json.load(f)


def map_kambi_market_key(input):
    match input.lower():
        case "match":
            return "h2h"
        case "handicap":
            return "spreads"
        case "over/under":
            return "totals"


# kambi_data = resp.json()
kambi_events = resp["events"]
print(f"Fetched {len(kambi_events)} events from Kambi")
print(f"Keys of first event: {list(kambi_events[0].keys())}")
print(f"Keys of event:{kambi_events[0]['event'].keys()}")
se = kambi_events[0]
se.update(**se.pop("event"))
del se["tags"]
del se["path"]
del se["nonLiveBoCount"]
del se["state"]
del se["englishName"]
# del se["participants"]
se.pop("extraInfo", None)
del se["name"]
del se["nameDelimiter"]
del se["groupId"]
se["commence_time"] = se.pop("start")
se["sport_key"] = se.pop("sport").strip("_").lower() + "_" + se["group"].lower()
se["sport_title"] = se.pop("group")
se["id"] = str(se.pop("id"))
se["home_team"] = se.pop("homeName")  # Normalize
se["away_team"] = se.pop("awayName")  # Normalize


markets = []
last_update = se["betOffers"][0]["outcomes"][0]["changedDate"]
for bo in se.pop("betOffers"):
    outcomes = []
    key = map_kambi_market_key(bo["betOfferType"]["name"])
    for o in bo["outcomes"]:
        outcome = {}
        if key == "h2h":
            outcome["name"] = o["participant"]
        else:
            outcome["name"] = o["label"]
            outcome["point"] = o["line"] / 1000
        outcome["price"] = o["odds"] / 1000
        outcome["last_update"] = o["changedDate"]
        outcomes.append(outcome)
    market = {
        "key": key,
        "last_update": bo["outcomes"][0]["changedDate"],
        "outcomes": outcomes,
    }
    markets.append(market)

bm = {
    "key": "kambi",
    "title": "KAMBI-CDN",
    "markets": markets,
    "last_update": last_update,
}
se["bookmakers"] = [bm]


outpath = "/home/maxo/code/oddstracker/data/se.json"

# print(f"Keys of betOffer:{kambi_events[0]['betOffer'].keys()}")
with open(outpath, "w") as f:
    json.dump(se, f, indent=2, default=str)


r = SportsEventInfo(**se)
r


Fetched 1 events from Kambi
Keys of first event: ['event', 'betOffers']
Keys of event:dict_keys(['id', 'name', 'nameDelimiter', 'englishName', 'homeName', 'awayName', 'start', 'group', 'groupId', 'path', 'nonLiveBoCount', 'sport', 'tags', 'state'])


SportsBettingInfo2(id='1024873535', sport_key='american_football_nfl', sport_title='NFL', commence_time='2025-10-03T00:15:00Z', home_team='LA Rams', away_team='SF 49ers', bookmakers=[Bookmaker(key='kambi', title='KAMBI-CDN', last_update='2025-10-02T22:14:40Z', markets=[Market(key='spreads', last_update='2025-10-02T22:14:40Z', outcomes=[Outcome(name='LA Rams', price=1.92, point=-8.5), Outcome(name='SF 49ers', price=1.9, point=8.5)]), Market(key='h2h', last_update='2025-10-02T15:14:19Z', outcomes=[Outcome(name='LA Rams', price=1.22, point=None), Outcome(name='SF 49ers', price=4.5, point=None)]), Market(key='totals', last_update='2025-10-02T22:14:40Z', outcomes=[Outcome(name='Under', price=1.93, point=44.0), Outcome(name='Over', price=1.89, point=44.0)])])])

In [ ]:
kambi_data = resp.json()
kambi_events = kambi_data["events"]
bos = kambi_events[0]['betOffers']
out = {}
bookmakers = [{'key':'kambi', 'title':'KAMBI-CDN','markets':[]}]
for bo in bos:
    boo = {**bo}

    # del boo["tags"]
    # del boo["cashOutStatus"]
    # del boo["closed"]


    bookmakers[0]['markets'].append(boo)

    # del bo["path"]
    # del bo["state"]
    # del bo["name"]
    # del bo["nameDelimiter"]
    # del bo["lastUpdate"]
    # outcomes = bo.pop("outcomes")
    # for outcome in outcomes:
    #     outcome.update(**outcome.pop("outcome"))
    #     del outcome["tags"]
    #     del outcome["path"]
    #     del outcome["state"]
    #     del outcome["nameDelimiter"]
    #     del outcome["lastUpdate"]
    # bo["outcomes"] = outcomes
out['bookmakers'] = bookmakers
with open(os.path.join(DATA_DIR, "bos2.json"), "w") as f:
    json.dump(out, f, indent=2, default=str)


In [ ]:
with open(os.path.join(DATA_DIR, "kdata.json"), "w", encoding="utf-8") as f:
    _test_data = kdata[1]
    out = ""
    for bo in _test_data.betOffers:
        out += bo.model_dump_json() + "\n"
    f.write(out)

In [ ]:
with open(os.path.join(DATA_DIR, "betOffers.json"), "w") as f:
    json.dump(kdata[0].betOffers[0].model_dump(), f, indent=2, default=str)


In [ ]:
import nflreadpy as nfl

# nfl.import_weekly_pfr('pass',[2025])
nfl.load_teams()

In [ ]:
import os

import requests

## https://github.com/the-odds-api/samples-python/blob/master/odds.py
## https://the-odds-api.com/liveapi/guides/v4/


# https://the-odds-api.com/sports-odds-data/sports-apis.html
SPORT = "americanfootball_nfl"

# decimal | american
ODDS_FORMAT = "decimal"

# Bookmaker regions
# uk | us | us2 | eu | au. Multiple can be specified if comma delimited.
# More info at https://the-odds-api.com/sports-odds-data/bookmaker-apis.html
REGIONS = "us2"
BOOKMAKER_KEY = "hardrockbet"

MARKETS = "h2h,spreads,totals"


API_KEY = os.environ.get("THEODDSAPI_KEY")
odds_response = requests.get(
    f"https://api.the-odds-api.com/v4/sports/{SPORT}/odds",
    params={
        "api_key": API_KEY,
        "regions": REGIONS,
        "markets": MARKETS,
        "oddsFormat": ODDS_FORMAT,
        "dateFormat": "iso",
    },
)

if odds_response.status_code != 200:
    print(
        f"Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}"
    )

else:
    odds_json = odds_response.json()
    print("Number of events:", len(odds_json))
    print(odds_json)

    # Check the usage quota
    print("Usage for request", odds_response.headers["x-requests-last"])
    print("Remaining requests", odds_response.headers["x-requests-remaining"])
    print("Used requests", odds_response.headers["x-requests-used"])